In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
import tensorflow as tf
from matplotlib import pyplot as plt
import torch.nn as nn
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from collections import OrderedDict
import torch.optim as optim
import warnings
warnings.filterwarnings("ignore")

In [ ]:
EV_Stations = pd.DataFrame(pd.read_csv('predicted_stations.csv'))
EV_Grid = pd.DataFrame(pd.read_csv('predicted_grid.csv'))

In [ ]:
EV_Stations = EV_Stations.drop(columns='Unnamed: 0')
EV_Stations = EV_Stations.loc[(EV_Stations['Year'] >= 2010) & (EV_Stations['Year'] <= 2022)]
EV_Stations 

In [ ]:
EV_Vehicles = pd.DataFrame(pd.read_csv('predicted_vehicles.csv'))
EV_Vehicles = EV_Vehicles.drop(columns='Unnamed: 0')
EV_Vehicles = EV_Vehicles.loc[(EV_Vehicles['Year'] >= 2010) & (EV_Vehicles['Year'] <= 2022)]
EV_Vehicles

In [ ]:
EV_Grid = EV_Grid.drop(columns='Unnamed: 0')
EV_Grid = EV_Grid.loc[(EV_Grid['Year'] >= 2010) & (EV_Grid['Year'] <= 2022)]
EV_Grid

In [ ]:
import math

# create a copy of the ev_grid array
ev_grid_scaled = EV_Grid.copy()

# select the predicted column
predicted_column = ev_grid_scaled['Predicted_MWH']

# scale the ev_grid value for all rows except the last row
predicted_column[:-1] = predicted_column[:-1] / math.exp(8)

# multiply the last row by e^7
#predicted_column[-1] = predicted_column[-1] * math.exp(7)
last_row = ev_grid_scaled.tail(1)
last_row = last_row / math.exp(7)
# change the last row of the predicted column
ev_grid_scaled.iloc[-1, ev_grid_scaled.columns.get_loc('Predicted_MWH')] = last_row
# update the predicted column in the ev_grid_scaled array
ev_grid_scaled['Predicted_MWH'] = predicted_column


ev_grid_scaled['Predicted_MWH'] = ev_grid_scaled['Predicted_MWH'].astype(int)
EV_Grid=ev_grid_scaled

EV_Grid

In [ ]:
print(EV_Vehicles.shape)
print(EV_Stations.shape)
print(EV_Grid.shape)

In [ ]:
# split the data into X (features) and y (target)
df = pd.merge(EV_Stations, EV_Grid, on='Year')
X = np.array(df[['Total_Installed', 'Predicted_MWH']])
y = np.array(EV_Stations['Year'])
print(X)
print(y)

In [ ]:
import torch
from torch import nn

num_inputs = 2
num_outputs = 1

hidden_size = [32, 64, 128, 256]

model = nn.Sequential(
nn.Linear(num_inputs, hidden_size[0]),
nn.ReLU(),
nn.Linear(hidden_size[0], hidden_size[1]),
nn.ReLU(),
nn.Linear(hidden_size[1], hidden_size[2]),
nn.ReLU(),
nn.Linear(hidden_size[2], hidden_size[3]),
nn.ReLU(),
nn.Linear(hidden_size[3],num_outputs),
)

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

# create an instance of the MinMaxScaler
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

y=y.reshape(-1,1)

# fit the scaler to the data
scaler_x.fit(X)
scaler_y.fit(y)

# transform the data
X = scaler_x.transform(X)
y = scaler_y.transform(y)

# normalize and scale the data
X_mean = X.mean()
X_std = X.std()
y_mean = y.mean()
y_std = y.std()

X = (X - X_mean) / X_std
y = (y - y_mean) / y_std
print(y.shape)
print(X.shape)
# split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# convert the data to tensors
X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()
y_train = torch.from_numpy(y_train).float()
y_test = torch.from_numpy(y_test).float()
print(X_train)
print(y_train)

In [ ]:

# define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)
print(model)
# train the model
for epoch in range(1000):
    # forward pass
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    # backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # print training and testing accuracy
    if epoch % 100 == 0:
        print(f'Epoch {epoch+1}/1000: Training loss = {loss.item():.4f}')
        with torch.no_grad():
            y_pred_test = model(X_test)
            test_loss = criterion(y_pred_test, y_test)
            print(f'Epoch {epoch+1}/1000: Testing loss = {test_loss.item():.4f}')

In [ ]:
# create a new data point
X_new = pd.DataFrame({'Year': [2023], 'EV_stations': [264.0]})

# convert the data to a tensor
X_new = torch.from_numpy(X_new.values).float()

# normalize and scale the data
#X_new = (X_new - X_mean) / X_std

# make predictions using the model
y_pred = model(X_new)

# de-normalize the prediction
y_pred = y_pred * y_std + y_mean

print(f'Predicted grid power for 2033 with 1574 EV stations: {y_pred.item():.4f}')

In [ ]:
# create a new data point
X_new = pd.DataFrame({'Year': [2023], 'EV_stations': [238.11287478]})

year = [2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033]
stations = [238.11287478,
 256.39153439, 274.670194,   292.94885362, 311.22751323, 329.50617284, 347.78483245, 366.06349206, 384.34215168, 402.62081129,
  420.8994709]


# convert the data to a tensor
X_new = torch.from_numpy(X_new.values).float()

# normalize and scale the data
#X_new = (X_new - X_mean) / X_std

# make predictions using the model
y_pred = model(X_new)

# de-normalize the prediction
y_pred = y_pred * y_std + y_mean

import math

# multiply the number by e^4
result = y_pred.item() * math.exp(5)
result
print(f'Predicted grid power for 2023 with 238 EV stations: {result:.4f}')

In [ ]:
import pandas as pd
import torch
import math

# Define the year and number of stations lists
year = [2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033]
stations = [238.11287478,
 256.39153439, 274.670194,   292.94885362, 311.22751323, 329.50617284, 347.78483245, 366.06349206, 384.34215168, 402.62081129,
  420.8994709]

# Create empty lists to store the predictions
predictions = []

# Iterate over the year and number of stations lists
for i in range(len(year)):
    # Create a new data point
    X_new = pd.DataFrame({'Year': [year[i]], 'EV_stations': [stations[i]]})

    # Convert the data to a tensor
    X_new = torch.from_numpy(X_new.values).float()

    # Normalize and scale the data
    #X_new = (X_new - X_mean) / X_std

    # Make predictions using the model
    y_pred = model(X_new)

    # De-normalize the prediction
    y_pred = y_pred * y_std + y_mean

    # Multiply the prediction by e^4
    result = y_pred * math.exp(5)
    
    # Add the prediction to the list
    predictions.append(result.item())

# Print the predictions
print(predictions)

In [ ]:
plt.plot(year,predictions)

In [ ]:
EV_Grid

In [ ]:
#x=X_train
#y=y_train
df = pd.DataFrame(pd.read_csv('annual_generation_WA.csv'))

x= np.arange(32) + 1990

y = df.Total.values


n = np.size(x)
predictions=predictions/math.exp(8)
fig=plt.figure(figsize = (15,6))
plt.title('Grid Capacity by Year')
plt.plot(x,y)
plt.plot(year,predictions)
plt.xlabel('Year')
plt.ylabel('Capacity in MWH')